### Prediction ###

In [1]:
from transformers import AutoModelForSequenceClassification, DistilBertTokenizer
import torch
import numpy as np
import torch.nn.functional as F

model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_model_2000")
tokenizer = DistilBertTokenizer.from_pretrained("./fine_tuned_model_2000")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


def tokenize_input(input_text):
    return tokenizer(
        input_text,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        max_length=128,
    )


def map_output(value: np.int64) -> str:
    mapping = {
        0: "negative",
        1: "positive",
    }
    return mapping.get(value, "???")


def predict(input_texts):
    model.eval()
    predictions = []

    for text in input_texts:
        inputs = tokenize_input(text)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        probs = F.softmax(logits, dim=1)
        predicted_class = np.argmax(probs.cpu().numpy(), axis=1)

        prediction_label = map_output(predicted_class[0])
        prediction_confidence = probs[0, predicted_class].item() * 100

        predictions.append((prediction_label, prediction_confidence))

    return predictions


new_texts = [
    "This movie was amazing! Don't you think?",
    "I hated this product. It's terrible.",
    "The book was okay, not great but not bad either.",
    "Neutral",
    "Everything",
]

predictions = predict(new_texts)

for text, (prediction, confidence) in zip(new_texts, predictions):
    print(f"text: {text}\nprediction: {prediction}\nconfidence: {confidence:.2f}%\n")


/home/buntish/.local/share/virtualenvs/Test_DL-u3DERucF/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-15 19:22:04.969598: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736965325.172952    3416 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736965325.234511    3416 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-15 19:22:05.689507: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructi

text: This movie was amazing! Don't you think?
prediction: positive
confidence: 99.88%

text: I hated this product. It's terrible.
prediction: negative
confidence: 99.93%

text: The book was okay, not great but not bad either.
prediction: positive
confidence: 99.31%

text: Neutral
prediction: negative
confidence: 99.84%

text: Everything
prediction: positive
confidence: 99.77%

